In [1]:
#import packages#
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import  train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
import lightgbm as lgb
import optuna
import optuna.integration.lightgbm as opt_lgb
from sklearn.metrics import log_loss, f1_score, matthews_corrcoef, roc_auc_score, confusion_matrix, brier_score_loss
import argparse
import joblib
from lightgbm import LGBMClassifier
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import optuna
import json
import gc
import shap
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
#training_helper#

def read_data(data_path):
    data = pd.read_csv(data_path)
    cat_features = list(data.select_dtypes(include=['object']).columns)
    data[cat_features] = data[cat_features].astype("category") 
    return data

def balanced_train_validation_test(data, target_column,random_state):
    X = data.loc[:, data.columns != target_column]
    y = data[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=random_state, stratify=y)
    return X_train, X_test, y_train, y_test

def fundtap_train_test_split(data,target_column = "label",
                     hold_out_columns = ["quote","fundtap_profit_loss"],
                    random_state = 456):
    
    categorical_feature = list(data.select_dtypes(include=['object']).columns)
    data[categorical_feature] = data[categorical_feature].astype("category")
    X_train, X_test, y_train, y_test = balanced_train_validation_test(data.loc[:, ~data.columns.isin(hold_out_columns)],target_column,random_state)
    
    train_instance_weight = np.abs(data.loc[X_train.index]["fundtap_profit_loss"])
    test_instance_weight = np.abs(data.loc[X_test.index]["fundtap_profit_loss"])

    return X_train, X_test, y_train, y_test, train_instance_weight, test_instance_weight




In [3]:
# Function to clean column names #
def clean_column_names(df):
    df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
    return df

# Assuming processed_data is a DataFrame
train_data = pd.read_csv('../data/train.csv')
processed_data = clean_column_names(train_data)

# Save the cleaned DataFrame to a CSV file
processed_data_path = '../data/processed_data.csv'
processed_data.to_csv(processed_data_path, index=False)


C:\Users\18741\AppData\Local\Temp\ipykernel_24284\1291651966.py:7: DtypeWarning: Columns (7,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('../data/train.csv')


In [7]:
# Load and process data
train_data_path = '../data/train.csv'
dictionary_path = '../data/fundtap-data-dictionary.csv'

train_data = read_data(train_data_path)
processed_data = clean_column_names(train_data)
processed_data["label"] = processed_data.fundtap_profit_loss >= 0

# Load data dictionary and select features for training
data_dictionary = pd.read_csv(dictionary_path)
columns_for_training = data_dictionary[data_dictionary["use_for_training"] == "Y"]["columns_cleaned"].tolist()

# Add label column to the selected columns
selected_columns = columns_for_training + ["label", "fundtap_profit_loss", "quote"]
df = processed_data[selected_columns]

# Run the training function
X_train, X_test, y_train, y_test, train_instance_weight, test_instance_weight = fundtap_train_test_split(
    df, 
    target_column="label",
    hold_out_columns=["quote", "fundtap_profit_loss"],
    random_state=456
)

# Output shapes of the splits and weights
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print("Train instance weight shape:", train_instance_weight.shape)
print("Test instance weight shape:", test_instance_weight.shape)
print("Number of columns used for training:", len(columns_for_training))

X_train shape: (6863, 142)
X_test shape: (1716, 142)
y_train shape: (6863,)
y_test shape: (1716,)
Train instance weight shape: (6863, 2)
Test instance weight shape: (1716, 2)
Number of columns used for training: 144


C:\Users\18741\AppData\Local\Temp\ipykernel_24284\4076922104.py:4: DtypeWarning: Columns (7,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)


Hi Eurus,
如上文：X_train 里现在有151个columns，y_train 里有1个colums. 所以参与模型training的columns一共有151+1=152个。
现在fundtap-data-dictionary（索引）里要求的use for training的columns一共却只有144个。
接下来你和gpt的任务，就是让实际参与模型training的columns数量 = use for training的columns
